<a href="https://colab.research.google.com/github/james-monahan/Code-school-notebooks/blob/main/%20Week-11-ml-classification-cluster%20/Challenge_KNN_classification_Pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#imports

In [1]:
! pip install pyforest

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13716 sha256=179f733a837412ae4a6b2d02580a8aab33d46abab3cdc46c464395f5196be5cd
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


In [ ]:
from pyforest import *

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get data

In [ ]:
url = 'https://drive.google.com/file/d/116tn0wRma8wpELBAGzgcsHYPC9uUcgti'
file_id = '116tn0wRma8wpELBAGzgcsHYPC9uUcgti'
downloaded = drive.CreateFile({'id': file_id}) 
print(downloaded['title']) 
downloaded.GetContentFile('pokemon.csv')  
df_poke = pd.read_csv('pokemon.csv') 

pokemon.csv


<IPython.core.display.Javascript object>

In [ ]:
df_poke.sample(3)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
98,99,Cloyster,Water,Ice,50,95,180,85,45,70,1,False
476,477,Mega Lopunny,Normal,Fighting,65,136,94,54,96,135,4,False
23,24,Mega Pidgeot,Normal,Flying,83,80,80,135,80,121,1,False


In [ ]:
df_poke.shape

(800, 12)

In [ ]:
df_poke.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

#preprocessing

In [ ]:
df = df_poke.copy()

In [ ]:
#df.isna().sum(), df.shape, df['Type 2'].value_counts()
df['Type 2'].fillna('missing', inplace=True)

In [ ]:
#there he is
df[df.isna().any(axis=1)]
df.dropna()[df.Name.dropna().str.contains('Giratina')]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
544,545,Giratina Altered Forme,Ghost,Dragon,150,100,120,100,120,90,4,True
545,546,Giratina Origin Forme,Ghost,Dragon,150,120,100,120,100,90,4,True


In [ ]:
#drop the one missing name
df = df.dropna()

In [ ]:
df.dtypes, df.shape

(#              int64
 Name          object
 Type 1        object
 Type 2        object
 HP             int64
 Attack         int64
 Defense        int64
 Sp. Atk        int64
 Sp. Def        int64
 Speed          int64
 Generation     int64
 Legendary       bool
 dtype: object, (799, 12))

In [ ]:
df['Type 1'] = df['Type 1'].astype('category')
df['Type 2'] = df['Type 2'].astype('category')

In [ ]:
df = pd.get_dummies(data=df, columns=['Type 1'], prefix='Type1', drop_first=True)
df = pd.get_dummies(data=df, columns=['Type 2'], prefix='Type2', drop_first=True)
df = pd.get_dummies(data=df, columns=['Legendary'], prefix='Legendary', drop_first=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
 df.shape

(799, 45)

#Classifier

In [ ]:
#some have multiple versions, I'm just going to grab the first for the model
pokes = ['Mewtwo', 'Lugia', 'Rayquaza', 'Giratina', 'Dialga', 'Palkia']
poke_index_ml = []

for name in pokes:
  poke_index = df[df.Name.str.contains(name)].index
  print(df.loc[poke_index][['#','Name']])
  poke_index_ml.append(poke_index[0])
# print()
# print("indices to use:",poke_index_ml)

       #           Name
162  163         Mewtwo
163  164  Mega Mewtwo X
164  165  Mega Mewtwo Y
       #   Name
269  270  Lugia
       #           Name
425  426       Rayquaza
426  427  Mega Rayquaza
       #                    Name
544  545  Giratina Altered Forme
545  546   Giratina Origin Forme
       #    Name
540  541  Dialga
       #    Name
541  542  Palkia


In [ ]:
df.head()

,#,Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Type1_Dark,Type1_Dragon,Type1_Electric,Type1_Fairy,Type1_Fighting,Type1_Fire,Type1_Flying,Type1_Ghost,Type1_Grass,Type1_Ground,Type1_Ice,Type1_Normal,Type1_Poison,Type1_Psychic,Type1_Rock,Type1_Steel,Type1_Water,Type2_Dark,Type2_Dragon,Type2_Electric,Type2_Fairy,Type2_Fighting,Type2_Fire,Type2_Flying,Type2_Ghost,Type2_Grass,Type2_Ground,Type2_Ice,Type2_Normal,Type2_Poison,Type2_Psychic,Type2_Rock,Type2_Steel,Type2_Water,Type2_missing,Legendary_True
0,1,Bulbasaur,45,49,49,65,65,45,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,Ivysaur,60,62,63,80,80,60,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3,Venusaur,80,82,83,100,100,80,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,Mega Venusaur,80,100,123,122,120,80,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,Charmander,39,52,43,60,50,65,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
def find_neighbors(data = df, neighbors = 1, poke = n, p=2):
  scaler = StandardScaler()

  single = data.loc[n]
  data = data[data['Legendary_True']==0]
  data = data.append(single)

  data_ = data.drop(['#','Name'], axis=1)
  data_scaled = scaler.fit_transform(data_)

  neigh = NearestNeighbors(n_neighbors=neighbors)
  neigh.fit(data_scaled)

  n_poke = neigh.kneighbors(data_scaled[-1:], return_distance=False)
  print("legendary",single['Name'])
  print(data.loc[n_poke[0][0], 'Name'])
  print(n_poke)
  

In [ ]:
#same replacelent for all, I can get other close neigbors, but not sure about instructions??
#is there supposed to be a unique replacement for all?

#this is printing addtional indexes of neighbors per each player requested
#in this case I can have a unique replacement for each player....maybe thats what is meant 
#by arguing for a k value, which would normally just be 1, the closest
for n in poke_index_ml:
  find_neighbors(data = df, neighbors = 5, poke = n, p=2)


legendary Mewtwo
Vivillon
[[734 158 204  70  69]]
legendary Lugia
Vivillon
[[734 578 185 507 158]]
legendary Rayquaza
Vivillon
[[734 396 397 157 353]]
legendary Giratina Altered Forme
Vivillon
[[734 242   7 263 189]]
legendary Dialga
Vivillon
[[734 242 263   7 189]]
legendary Palkia
Vivillon
[[734 242 263   7 189]]


In [ ]:
df_poke[df_poke.Name == 'Vivillon']

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
734,735,Vivillon,Bug,Flying,80,52,50,90,50,89,6,False
